In [45]:
import pandas as pd 
from pathlib import Path 
from tqdm import tqdm

To start off with I have exported one of the files to an excel sheet. We need to understand the contents of each recording. How the columns differ in each sheet (represent this with a tree diagram?). Then I would like to export each of these sheets as a numpy file as it takes a long time to load. 

In [46]:
example_file = Path("PD046-OFF_Falling_GDPURS.xlsx") # Loaded in the example file 

# pandas requires the sheet name as an argument, otherwise it will load the first one by default. I have gone through the sheet and copied the names to this array. 
sheet_names = ["General Information", "Segment Orientation - Quat", "Segment Orientation - Euler", "Segment Position", "Segment Velocity", "Segment Acceleration", "Segment Angular Velocity", "Segment Angular Acceleration", "Joint Angles ZXY", "Joint Angles XZY", "Ergonomic Joint Angles ZXY", "Ergonomic Joint Angles XZY", "Center of Mass", "Sensor Free Acceleration", "Sensor Magnetic Field", "Sensor Orientation - Quat", "Sensor Orientation - Euler"]

In [47]:
def columns2md(file_path,sheet_name):

    data_df = pd.read_excel(file_path, sheet_name=sheet_name)

    split_column_names = pd.DataFrame.from_records((x.split(" ") for x in data_df.columns))

    def add_to_dict(x, dictionary):
        if len(x) == 1:
            if x[0] not in dictionary.keys():
                dictionary[x[0]] = []
            return
            
        if len(x) > 2:
            if x[0] not in dictionary.keys():
                dictionary[x[0]] = {}
            add_to_dict(x[1:], dictionary[x[0]])
        else:
            if x[0] not in dictionary.keys():
                dictionary[x[0]] = []
            dictionary[x[0]].append(x[1])
            
    column_tree = {}
    split_column_names.T.apply(lambda x: add_to_dict(x.dropna().values, column_tree))

    def write_dict(dictionary, prefix, file):
        for name, item in dictionary.items():
            file.write(prefix + "- " + name + "\n")
            if type(item) == dict:
                write_dict(item, prefix + "\t", file)
            elif len(item) > 0:
                file.write(prefix + "\t" + "- " + ", ".join(item) + "\n")

    with open(sheetname + ".md", 'w') as f:
        f.write("# " + sheetname + "\n")
        write_dict(column_tree, "", f)

In [48]:
for sheet_name in tqdm(sheet_names):
    columns2md(data_df, sheet_name)

 29%|██▉       | 5/17 [02:28<05:56, 29.70s/it]


KeyboardInterrupt: 

In [ ]:
from multiprocessing import Pool, cpu_count

with Pool() as pool:
        res = pool.map(columns2md, values)
        print(res)